# Preparação

Temos um desáfio que é aumentar nossos números no próximo periodo e precisamo, com nosso conjunto de dados avaliar e observar se existe a possibilidade de adicionarmos mais inferências que irão nos ajudar nessa meta!

Meu objetivo é avaliar os dados existentes e criar novas inferências para melhor tomada de decisão orientada por dados.

In [1]:
# Libs
import pandas as pd

# Modulos
from e_commerce.data_extraction.extractor import get_data
from e_commerce.utils.save import save_to_csv_processed

In [2]:
input_path = 'e_commerce_clean.csv'
df_preparation = get_data(source_type='csv', type_name='interim', filename_or_path=input_path)
df_preparation

,order_date,lead_time,customer_id,gender,device_type,customer_login_type,product_category,product,sales,quantity,discount,profit,shipping_cost,order_priority,payment_method
0,2018-01-02 10:56:33,8,37077,Female,Web,Member,Auto & Accessories,Car Media Players,140.0,1,0.3,46.0,4.6,Medium,credit_card
1,2018-07-24 20:41:37,2,59173,Female,Web,Member,Auto & Accessories,Car Speakers,211.0,1,0.3,112.0,11.2,Medium,credit_card
2,2018-11-08 08:38:49,8,41066,Female,Web,Member,Auto & Accessories,Car Body Covers,117.0,5,0.1,31.2,3.1,Critical,credit_card
3,2018-04-18 19:28:06,7,50741,Female,Web,Member,Auto & Accessories,Car & Bike Care,118.0,1,0.3,26.2,2.6,High,credit_card
4,2018-08-13 21:18:39,9,53639,Female,Web,Member,Auto & Accessories,Tyre,250.0,1,0.3,160.0,16.0,Critical,credit_card
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51279,2018-02-28 22:59:50,6,78489,Female,Mobile,Member,Home & Furniture,Shoe Rack,124.0,4,0.3,19.2,1.9,Medium,money_order
51280,2018-02-28 13:19:25,2,91941,Female,Mobile,Member,Home & Furniture,Umbrellas,70.0,5,0.2,14.0,1.4,Medium,credit_card
51281,2018-02-28 10:25:07,6,63313,Male,Web,Member,Home & Furniture,Dinner Crockery,133.0,1,0.3,39.7,4.0,Medium,credit_card
51282,2018-02-28 10:50:08,7,86485,Male,Web,Member,Home & Furniture,Sofa Covers,216.0,1,0.2,131.7,13.2,Medium,credit_card


In [3]:
df_preparation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51284 entries, 0 to 51283
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   order_date           51284 non-null  object 
 1   lead_time            51284 non-null  int64  
 2   customer_id          51284 non-null  int64  
 3   gender               51284 non-null  object 
 4   device_type          51284 non-null  object 
 5   customer_login_type  51284 non-null  object 
 6   product_category     51284 non-null  object 
 7   product              51284 non-null  object 
 8   sales                51284 non-null  float64
 9   quantity             51284 non-null  int64  
 10  discount             51284 non-null  float64
 11  profit               51284 non-null  float64
 12  shipping_cost        51284 non-null  float64
 13  order_priority       51284 non-null  object 
 14  payment_method       51284 non-null  object 
dtypes: float64(4), int64(3), object(8)
m

In [4]:
df_preparation['order_date'] = pd.to_datetime(df_preparation['order_date'])
df_preparation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51284 entries, 0 to 51283
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   order_date           51284 non-null  datetime64[ns]
 1   lead_time            51284 non-null  int64         
 2   customer_id          51284 non-null  int64         
 3   gender               51284 non-null  object        
 4   device_type          51284 non-null  object        
 5   customer_login_type  51284 non-null  object        
 6   product_category     51284 non-null  object        
 7   product              51284 non-null  object        
 8   sales                51284 non-null  float64       
 9   quantity             51284 non-null  int64         
 10  discount             51284 non-null  float64       
 11  profit               51284 non-null  float64       
 12  shipping_cost        51284 non-null  float64       
 13  order_priority       51284 non-

In [5]:
# Averigua clientes que mas compraram por genero
df_preparation.groupby(['customer_id', 'gender'])['sales'].count().reset_index().sort_values(by='sales',ascending=False)

,customer_id,gender,sales
30097,75238,Female,5
34808,83647,Female,5
37040,87562,Female,5
39881,92705,Female,5
3080,16997,Male,4
...,...,...,...
44011,99990,Female,1
44012,99991,Male,1
44013,99992,Female,1
44014,99993,Male,1


In [6]:
print(f"Menor tempo na entrega: {df_preparation['lead_time'].min()}")
print(f"Maior tempo na entrega: {df_preparation['lead_time'].max()}")
print(f"Tempo médio de entrega: {df_preparation['lead_time'].mean().round().astype(int)}")

Menor tempo na entrega: 1
Maior tempo na entrega: 10
Tempo médio de entrega: 5


In [7]:
# Cria coluna da data de entrega estimada
data_entrega = pd.to_datetime(df_preparation['order_date'].dt.date + pd.Timedelta(days=8))
df_preparation.insert(1, 'estimated_delivery_date', data_entrega)
df_preparation

,order_date,estimated_delivery_date,lead_time,customer_id,gender,device_type,customer_login_type,product_category,product,sales,quantity,discount,profit,shipping_cost,order_priority,payment_method
0,2018-01-02 10:56:33,2018-01-10,8,37077,Female,Web,Member,Auto & Accessories,Car Media Players,140.0,1,0.3,46.0,4.6,Medium,credit_card
1,2018-07-24 20:41:37,2018-08-01,2,59173,Female,Web,Member,Auto & Accessories,Car Speakers,211.0,1,0.3,112.0,11.2,Medium,credit_card
2,2018-11-08 08:38:49,2018-11-16,8,41066,Female,Web,Member,Auto & Accessories,Car Body Covers,117.0,5,0.1,31.2,3.1,Critical,credit_card
3,2018-04-18 19:28:06,2018-04-26,7,50741,Female,Web,Member,Auto & Accessories,Car & Bike Care,118.0,1,0.3,26.2,2.6,High,credit_card
4,2018-08-13 21:18:39,2018-08-21,9,53639,Female,Web,Member,Auto & Accessories,Tyre,250.0,1,0.3,160.0,16.0,Critical,credit_card
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51279,2018-02-28 22:59:50,2018-03-08,6,78489,Female,Mobile,Member,Home & Furniture,Shoe Rack,124.0,4,0.3,19.2,1.9,Medium,money_order
51280,2018-02-28 13:19:25,2018-03-08,2,91941,Female,Mobile,Member,Home & Furniture,Umbrellas,70.0,5,0.2,14.0,1.4,Medium,credit_card
51281,2018-02-28 10:25:07,2018-03-08,6,63313,Male,Web,Member,Home & Furniture,Dinner Crockery,133.0,1,0.3,39.7,4.0,Medium,credit_card
51282,2018-02-28 10:50:08,2018-03-08,7,86485,Male,Web,Member,Home & Furniture,Sofa Covers,216.0,1,0.2,131.7,13.2,Medium,credit_card


In [8]:
# Cria coluna da data de entrega
data_entrega = pd.to_datetime(df_preparation['order_date'].dt.date + pd.to_timedelta(df_preparation['lead_time'], unit='D'))
df_preparation.insert(2, 'delivery_date', data_entrega)
df_preparation

C:\Users\jacom\AppData\Local\Temp\ipykernel_13680\1428686875.py:2: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  data_entrega = pd.to_datetime(df_preparation['order_date'].dt.date + pd.to_timedelta(df_preparation['lead_time'], unit='D'))


,order_date,estimated_delivery_date,delivery_date,lead_time,customer_id,gender,device_type,customer_login_type,product_category,product,sales,quantity,discount,profit,shipping_cost,order_priority,payment_method
0,2018-01-02 10:56:33,2018-01-10,2018-01-10,8,37077,Female,Web,Member,Auto & Accessories,Car Media Players,140.0,1,0.3,46.0,4.6,Medium,credit_card
1,2018-07-24 20:41:37,2018-08-01,2018-07-26,2,59173,Female,Web,Member,Auto & Accessories,Car Speakers,211.0,1,0.3,112.0,11.2,Medium,credit_card
2,2018-11-08 08:38:49,2018-11-16,2018-11-16,8,41066,Female,Web,Member,Auto & Accessories,Car Body Covers,117.0,5,0.1,31.2,3.1,Critical,credit_card
3,2018-04-18 19:28:06,2018-04-26,2018-04-25,7,50741,Female,Web,Member,Auto & Accessories,Car & Bike Care,118.0,1,0.3,26.2,2.6,High,credit_card
4,2018-08-13 21:18:39,2018-08-21,2018-08-22,9,53639,Female,Web,Member,Auto & Accessories,Tyre,250.0,1,0.3,160.0,16.0,Critical,credit_card
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51279,2018-02-28 22:59:50,2018-03-08,2018-03-06,6,78489,Female,Mobile,Member,Home & Furniture,Shoe Rack,124.0,4,0.3,19.2,1.9,Medium,money_order
51280,2018-02-28 13:19:25,2018-03-08,2018-03-02,2,91941,Female,Mobile,Member,Home & Furniture,Umbrellas,70.0,5,0.2,14.0,1.4,Medium,credit_card
51281,2018-02-28 10:25:07,2018-03-08,2018-03-06,6,63313,Male,Web,Member,Home & Furniture,Dinner Crockery,133.0,1,0.3,39.7,4.0,Medium,credit_card
51282,2018-02-28 10:50:08,2018-03-08,2018-03-07,7,86485,Male,Web,Member,Home & Furniture,Sofa Covers,216.0,1,0.2,131.7,13.2,Medium,credit_card


In [9]:
# Cria coluna de atraso na entrega
df_preparation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51284 entries, 0 to 51283
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   order_date               51284 non-null  datetime64[ns]
 1   estimated_delivery_date  51284 non-null  datetime64[ns]
 2   delivery_date            51284 non-null  datetime64[ns]
 3   lead_time                51284 non-null  int64         
 4   customer_id              51284 non-null  int64         
 5   gender                   51284 non-null  object        
 6   device_type              51284 non-null  object        
 7   customer_login_type      51284 non-null  object        
 8   product_category         51284 non-null  object        
 9   product                  51284 non-null  object        
 10  sales                    51284 non-null  float64       
 11  quantity                 51284 non-null  int64         
 12  discount                 51284 n

In [10]:
dias_entrega = df_preparation['lead_time']
status_entrega = pd.DataFrame(["Delayed" if dia > 8 else "On Time" for dia in dias_entrega])
status_entrega

,0
0,On Time
1,On Time
2,On Time
3,On Time
4,Delayed
...,...
51279,On Time
51280,On Time
51281,On Time
51282,On Time


In [11]:
# Cria coluna status de entrega coluna criada para simular produtos atrasados e dar mais vida a análise
df_preparation.insert(4, 'shipping_status', status_entrega)
df_preparation

,order_date,estimated_delivery_date,delivery_date,lead_time,shipping_status,customer_id,gender,device_type,customer_login_type,product_category,product,sales,quantity,discount,profit,shipping_cost,order_priority,payment_method
0,2018-01-02 10:56:33,2018-01-10,2018-01-10,8,On Time,37077,Female,Web,Member,Auto & Accessories,Car Media Players,140.0,1,0.3,46.0,4.6,Medium,credit_card
1,2018-07-24 20:41:37,2018-08-01,2018-07-26,2,On Time,59173,Female,Web,Member,Auto & Accessories,Car Speakers,211.0,1,0.3,112.0,11.2,Medium,credit_card
2,2018-11-08 08:38:49,2018-11-16,2018-11-16,8,On Time,41066,Female,Web,Member,Auto & Accessories,Car Body Covers,117.0,5,0.1,31.2,3.1,Critical,credit_card
3,2018-04-18 19:28:06,2018-04-26,2018-04-25,7,On Time,50741,Female,Web,Member,Auto & Accessories,Car & Bike Care,118.0,1,0.3,26.2,2.6,High,credit_card
4,2018-08-13 21:18:39,2018-08-21,2018-08-22,9,Delayed,53639,Female,Web,Member,Auto & Accessories,Tyre,250.0,1,0.3,160.0,16.0,Critical,credit_card
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51279,2018-02-28 22:59:50,2018-03-08,2018-03-06,6,On Time,78489,Female,Mobile,Member,Home & Furniture,Shoe Rack,124.0,4,0.3,19.2,1.9,Medium,money_order
51280,2018-02-28 13:19:25,2018-03-08,2018-03-02,2,On Time,91941,Female,Mobile,Member,Home & Furniture,Umbrellas,70.0,5,0.2,14.0,1.4,Medium,credit_card
51281,2018-02-28 10:25:07,2018-03-08,2018-03-06,6,On Time,63313,Male,Web,Member,Home & Furniture,Dinner Crockery,133.0,1,0.3,39.7,4.0,Medium,credit_card
51282,2018-02-28 10:50:08,2018-03-08,2018-03-07,7,On Time,86485,Male,Web,Member,Home & Furniture,Sofa Covers,216.0,1,0.2,131.7,13.2,Medium,credit_card


In [12]:
# Cria coluna faturamento bruto ou total bruto
faturamento_bruto= (df_preparation['sales'] * df_preparation['quantity'])
df_preparation.insert(13, 'gross_revenue', faturamento_bruto)

In [13]:
df_preparation

,order_date,estimated_delivery_date,delivery_date,lead_time,shipping_status,customer_id,gender,device_type,customer_login_type,product_category,product,sales,quantity,gross_revenue,discount,profit,shipping_cost,order_priority,payment_method
0,2018-01-02 10:56:33,2018-01-10,2018-01-10,8,On Time,37077,Female,Web,Member,Auto & Accessories,Car Media Players,140.0,1,140.0,0.3,46.0,4.6,Medium,credit_card
1,2018-07-24 20:41:37,2018-08-01,2018-07-26,2,On Time,59173,Female,Web,Member,Auto & Accessories,Car Speakers,211.0,1,211.0,0.3,112.0,11.2,Medium,credit_card
2,2018-11-08 08:38:49,2018-11-16,2018-11-16,8,On Time,41066,Female,Web,Member,Auto & Accessories,Car Body Covers,117.0,5,585.0,0.1,31.2,3.1,Critical,credit_card
3,2018-04-18 19:28:06,2018-04-26,2018-04-25,7,On Time,50741,Female,Web,Member,Auto & Accessories,Car & Bike Care,118.0,1,118.0,0.3,26.2,2.6,High,credit_card
4,2018-08-13 21:18:39,2018-08-21,2018-08-22,9,Delayed,53639,Female,Web,Member,Auto & Accessories,Tyre,250.0,1,250.0,0.3,160.0,16.0,Critical,credit_card
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51279,2018-02-28 22:59:50,2018-03-08,2018-03-06,6,On Time,78489,Female,Mobile,Member,Home & Furniture,Shoe Rack,124.0,4,496.0,0.3,19.2,1.9,Medium,money_order
51280,2018-02-28 13:19:25,2018-03-08,2018-03-02,2,On Time,91941,Female,Mobile,Member,Home & Furniture,Umbrellas,70.0,5,350.0,0.2,14.0,1.4,Medium,credit_card
51281,2018-02-28 10:25:07,2018-03-08,2018-03-06,6,On Time,63313,Male,Web,Member,Home & Furniture,Dinner Crockery,133.0,1,133.0,0.3,39.7,4.0,Medium,credit_card
51282,2018-02-28 10:50:08,2018-03-08,2018-03-07,7,On Time,86485,Male,Web,Member,Home & Furniture,Sofa Covers,216.0,1,216.0,0.2,131.7,13.2,Medium,credit_card


In [15]:
# Cria coluna faturamento liquido ou total liquido
faturamento_liquido = round(df_preparation['gross_revenue'] - (df_preparation['gross_revenue'] * df_preparation['discount']), 2)
df_preparation.insert(15, 'net_total', faturamento_liquido)
df_preparation

,order_date,estimated_delivery_date,delivery_date,lead_time,shipping_status,customer_id,gender,device_type,customer_login_type,product_category,product,sales,quantity,gross_revenue,discount,net_total,profit,shipping_cost,order_priority,payment_method
0,2018-01-02 10:56:33,2018-01-10,2018-01-10,8,On Time,37077,Female,Web,Member,Auto & Accessories,Car Media Players,140.0,1,140.0,0.3,98.0,46.0,4.6,Medium,credit_card
1,2018-07-24 20:41:37,2018-08-01,2018-07-26,2,On Time,59173,Female,Web,Member,Auto & Accessories,Car Speakers,211.0,1,211.0,0.3,147.7,112.0,11.2,Medium,credit_card
2,2018-11-08 08:38:49,2018-11-16,2018-11-16,8,On Time,41066,Female,Web,Member,Auto & Accessories,Car Body Covers,117.0,5,585.0,0.1,526.5,31.2,3.1,Critical,credit_card
3,2018-04-18 19:28:06,2018-04-26,2018-04-25,7,On Time,50741,Female,Web,Member,Auto & Accessories,Car & Bike Care,118.0,1,118.0,0.3,82.6,26.2,2.6,High,credit_card
4,2018-08-13 21:18:39,2018-08-21,2018-08-22,9,Delayed,53639,Female,Web,Member,Auto & Accessories,Tyre,250.0,1,250.0,0.3,175.0,160.0,16.0,Critical,credit_card
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51279,2018-02-28 22:59:50,2018-03-08,2018-03-06,6,On Time,78489,Female,Mobile,Member,Home & Furniture,Shoe Rack,124.0,4,496.0,0.3,347.2,19.2,1.9,Medium,money_order
51280,2018-02-28 13:19:25,2018-03-08,2018-03-02,2,On Time,91941,Female,Mobile,Member,Home & Furniture,Umbrellas,70.0,5,350.0,0.2,280.0,14.0,1.4,Medium,credit_card
51281,2018-02-28 10:25:07,2018-03-08,2018-03-06,6,On Time,63313,Male,Web,Member,Home & Furniture,Dinner Crockery,133.0,1,133.0,0.3,93.1,39.7,4.0,Medium,credit_card
51282,2018-02-28 10:50:08,2018-03-08,2018-03-07,7,On Time,86485,Male,Web,Member,Home & Furniture,Sofa Covers,216.0,1,216.0,0.2,172.8,131.7,13.2,Medium,credit_card


In [16]:
df_preparation.columns

Index(['order_date', 'estimated_delivery_date', 'delivery_date', 'lead_time',
       'shipping_status', 'customer_id', 'gender', 'device_type',
       'customer_login_type', 'product_category', 'product', 'sales',
       'quantity', 'gross_revenue', 'discount', 'net_total', 'profit',
       'shipping_cost', 'order_priority', 'payment_method'],
      dtype='object')

In [17]:
# Cria coluna receita liquida
faturamento_liquido = round(df_preparation['net_total'] - df_preparation['shipping_cost'])
df_preparation.insert(16, 'net_revenue', faturamento_liquido)
df_preparation

,order_date,estimated_delivery_date,delivery_date,lead_time,shipping_status,customer_id,gender,device_type,customer_login_type,product_category,...,sales,quantity,gross_revenue,discount,net_total,net_revenue,profit,shipping_cost,order_priority,payment_method
0,2018-01-02 10:56:33,2018-01-10,2018-01-10,8,On Time,37077,Female,Web,Member,Auto & Accessories,...,140.0,1,140.0,0.3,98.0,93.0,46.0,4.6,Medium,credit_card
1,2018-07-24 20:41:37,2018-08-01,2018-07-26,2,On Time,59173,Female,Web,Member,Auto & Accessories,...,211.0,1,211.0,0.3,147.7,136.0,112.0,11.2,Medium,credit_card
2,2018-11-08 08:38:49,2018-11-16,2018-11-16,8,On Time,41066,Female,Web,Member,Auto & Accessories,...,117.0,5,585.0,0.1,526.5,523.0,31.2,3.1,Critical,credit_card
3,2018-04-18 19:28:06,2018-04-26,2018-04-25,7,On Time,50741,Female,Web,Member,Auto & Accessories,...,118.0,1,118.0,0.3,82.6,80.0,26.2,2.6,High,credit_card
4,2018-08-13 21:18:39,2018-08-21,2018-08-22,9,Delayed,53639,Female,Web,Member,Auto & Accessories,...,250.0,1,250.0,0.3,175.0,159.0,160.0,16.0,Critical,credit_card
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51279,2018-02-28 22:59:50,2018-03-08,2018-03-06,6,On Time,78489,Female,Mobile,Member,Home & Furniture,...,124.0,4,496.0,0.3,347.2,345.0,19.2,1.9,Medium,money_order
51280,2018-02-28 13:19:25,2018-03-08,2018-03-02,2,On Time,91941,Female,Mobile,Member,Home & Furniture,...,70.0,5,350.0,0.2,280.0,279.0,14.0,1.4,Medium,credit_card
51281,2018-02-28 10:25:07,2018-03-08,2018-03-06,6,On Time,63313,Male,Web,Member,Home & Furniture,...,133.0,1,133.0,0.3,93.1,89.0,39.7,4.0,Medium,credit_card
51282,2018-02-28 10:50:08,2018-03-08,2018-03-07,7,On Time,86485,Male,Web,Member,Home & Furniture,...,216.0,1,216.0,0.2,172.8,160.0,131.7,13.2,Medium,credit_card


In [20]:
# Cria coluna custo diferenciado para evitar conclusões erradas sobre impostos indevidos
diferenca_custo = df_preparation['net_revenue'] - df_preparation['profit']
df_preparation.insert(17, 'cost_difference', diferenca_custo)
df_preparation

,order_date,estimated_delivery_date,delivery_date,lead_time,shipping_status,customer_id,gender,device_type,customer_login_type,product_category,...,quantity,gross_revenue,discount,net_total,net_revenue,cost_difference,profit,shipping_cost,order_priority,payment_method
0,2018-01-02 10:56:33,2018-01-10,2018-01-10,8,On Time,37077,Female,Web,Member,Auto & Accessories,...,1,140.0,0.3,98.0,93.0,47.0,46.0,4.6,Medium,credit_card
1,2018-07-24 20:41:37,2018-08-01,2018-07-26,2,On Time,59173,Female,Web,Member,Auto & Accessories,...,1,211.0,0.3,147.7,136.0,24.0,112.0,11.2,Medium,credit_card
2,2018-11-08 08:38:49,2018-11-16,2018-11-16,8,On Time,41066,Female,Web,Member,Auto & Accessories,...,5,585.0,0.1,526.5,523.0,491.8,31.2,3.1,Critical,credit_card
3,2018-04-18 19:28:06,2018-04-26,2018-04-25,7,On Time,50741,Female,Web,Member,Auto & Accessories,...,1,118.0,0.3,82.6,80.0,53.8,26.2,2.6,High,credit_card
4,2018-08-13 21:18:39,2018-08-21,2018-08-22,9,Delayed,53639,Female,Web,Member,Auto & Accessories,...,1,250.0,0.3,175.0,159.0,-1.0,160.0,16.0,Critical,credit_card
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51279,2018-02-28 22:59:50,2018-03-08,2018-03-06,6,On Time,78489,Female,Mobile,Member,Home & Furniture,...,4,496.0,0.3,347.2,345.0,325.8,19.2,1.9,Medium,money_order
51280,2018-02-28 13:19:25,2018-03-08,2018-03-02,2,On Time,91941,Female,Mobile,Member,Home & Furniture,...,5,350.0,0.2,280.0,279.0,265.0,14.0,1.4,Medium,credit_card
51281,2018-02-28 10:25:07,2018-03-08,2018-03-06,6,On Time,63313,Male,Web,Member,Home & Furniture,...,1,133.0,0.3,93.1,89.0,49.3,39.7,4.0,Medium,credit_card
51282,2018-02-28 10:50:08,2018-03-08,2018-03-07,7,On Time,86485,Male,Web,Member,Home & Furniture,...,1,216.0,0.2,172.8,160.0,28.3,131.7,13.2,Medium,credit_card


### Salvar dataset processado para análise

In [21]:
save_to_csv_processed(df_preparation, filename='e_commerce_prepared.csv', index=False)

✅ Arquivo salvo em: H:\Portifolios_e_anotacoes_Jackson\PORTIFÓLIOS\america_ecommerce\data\processed\e_commerce_prepared.csv


'H:\\Portifolios_e_anotacoes_Jackson\\PORTIFÓLIOS\\america_ecommerce\\data\\processed\\e_commerce_prepared.csv'